##E-commerce Data Pipeline and Predictive Modeling Project

#Introduction

The goal of this project was to transform raw, disparate sales and inventory data from five separate Amazon reports into a single, high-fidelity dataset suitable for Machine Learning (ML). The pipeline covered data integration, deep cleaning, advanced feature engineering, exploratory data analysis (EDA), and final model development using a classification algorithm to predict a key business outcome. The final output is an ML-ready dataset of over 130,000 records and 93 features, coupled with an optimized predictive model.

###Phase 1:
Data Integration and Core Feature Engineering
This phase focused on building the foundational data asset by combining sales and catalog reports and standardizing key identifiers.


###Work Steps
Data Ingestion and Unification (ETL): Loaded the core Domestic Sales Report (128,975 records) and the International Sales Report, along with two versions of the product catalog and a stock report.


Vertical Concatenation: Joined the domestic and international sales records into a single fact_all table using a vertical concatenation (pd.concat).

SKU Feature Engineering: Parsed the raw SKU string to create three distinct, clean identifiers: SKU_ID, SKU_SIZE, and the combined primary key, SKU_ID_SIZE, to ensure consistent merging across tables.

Temporal Merge Strategy: Implemented conditional merging logic to link the sales data with the correct product catalog version (March vs. May) based on the transaction date, preserving the accuracy of historical product attributes.

Initial Cleaning: Filtered sales to only include successful transactions ("Shipped - Delivered to Buyer") with Qty > 0.

##Conclusion & Example Results

The data was successfully unified into a single transactional table containing all relevant sales.

Result: The initial sales table was reduced from 128,975 records to ~97,621 completed and valid transactions.

Example: Successfully engineered the SKU_ID_SIZE primary key, which was critical for performing accurate Left Join operations in subsequent steps.

In [ ]:
import pandas as pd

# DATA PREPARATION




### 1. Uploading the data

In [ ]:
Amazon_Report  = pd.read_csv(r"Amazon Sale Report.csv")
Amazon_Report.info()
Amazon_Report.head()

/tmp/ipython-input-635453339.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  Amazon_Report  = pd.read_csv(r"Amazon Sale Report.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128975 non-null  int64  
 1   Order ID            128975 non-null  object 
 2   Date                128975 non-null  object 
 3   Status              128975 non-null  object 
 4   Fulfilment          128975 non-null  object 
 5   Sales Channel       128975 non-null  object 
 6   ship-service-level  128975 non-null  object 
 7   Style               128975 non-null  object 
 8   SKU                 128975 non-null  object 
 9   Category            128975 non-null  object 
 10  Size                128975 non-null  object 
 11  ASIN                128975 non-null  object 
 12  Courier Status      122103 non-null  object 
 13  Qty                 128975 non-null  int64  
 14  currency            121180 non-null  object 
 15  Amount              121180 non-nul

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


In [ ]:
# Display unique SKU values
Amazon_Report.SKU.unique()

# Extract the first part before the first hyphen ('-')
Amazon_Report["SKU_ID"] = Amazon_Report["SKU"].str.split("-", n=1).str[0]

# Extract the last part after the last hyphen ('-')
Amazon_Report["SKU_SIZE"] = Amazon_Report["SKU"].str.rsplit("-", n=1).str[-1]

# Create a combined column of SKU_ID and SKU_SIZE
Amazon_Report["SKU_ID_SIZE"] = Amazon_Report["SKU_ID"] + "-" + Amazon_Report["SKU_SIZE"]

# Print the updated DataFrame to verify changes
print(Amazon_Report)

# Display detailed information about the DataFrame (columns, data types, memory usage, etc.)
Amazon_Report.info()


         index             Order ID      Date                        Status  \
0            0  405-8078784-5731545  04-30-22                     Cancelled   
1            1  171-9198151-1101146  04-30-22  Shipped - Delivered to Buyer   
2            2  404-0687676-7273146  04-30-22                       Shipped   
3            3  403-9615377-8133951  04-30-22                     Cancelled   
4            4  407-1069790-7240320  04-30-22                       Shipped   
...        ...                  ...       ...                           ...   
128970  128970  406-6001380-7673107  05-31-22                       Shipped   
128971  128971  402-9551604-7544318  05-31-22                       Shipped   
128972  128972  407-9547469-3152358  05-31-22                       Shipped   
128973  128973  402-6184140-0545956  05-31-22                       Shipped   
128974  128974  408-7436540-8728312  05-31-22                       Shipped   

       Fulfilment Sales Channel  ship-service-level

In [ ]:
# Display unique values in 'ship-country' column (e.g., 'IN', NaN)
Amazon_Report['ship-country'].unique()  # array(['IN', nan], dtype=object)

# Display unique dates in the 'Date' column (range: 30.3.22 to 29.6.22)
Amazon_Report['Date'].unique()

# Display value counts for 'SKU' (total 7,195 unique values, ranging from 773 to 1)
Amazon_Report['SKU'].value_counts()

# Convert 'SKU' column to string type for text processing
s = Amazon_Report["SKU"].astype("string")

# Condition: keep only values that contain at least one letter and one digit (case-insensitive)
mask = s.str.contains(r"(?i)(?=.*[A-Za-z])(?=.*\d)", na=False)

# Create 'SKU_clean' column: keep valid values only, convert them to uppercase
Amazon_Report["SKU_clean"] = s.where(mask).str.upper()

# Quick validation check (optional)
print(Amazon_Report["SKU_clean"].value_counts(dropna=False).head(20))
print("Kept:", mask.sum(), "| Dropped:", (~mask).sum())

# Display DataFrame information (column names, types, non-null counts, etc.)
Amazon_Report.info()


SKU_clean
JNE3797-KR-L       773
JNE3797-KR-M       657
JNE3797-KR-S       587
JNE3405-KR-L       535
J0230-SKD-M        507
JNE3797-KR-XL      474
J0230-SKD-S        452
JNE3405-KR-S       443
JNE3797-KR-XS      431
JNE3797-KR-XXL     395
JNE3405-KR-M       394
SET268-KR-NP-S     387
JNE3797-KR-XXXL    386
SET268-KR-NP-XL    386
SET268-KR-NP-L     362
SET183-KR-DH-M     343
J0003-SET-S        334
JNE3567-KR-M       317
SET324-KR-NP-M     305
J0230-SKD-L        297
Name: count, dtype: Int64
Kept: 128975 | Dropped: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 28 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128975 non-null  int64  
 1   Order ID            128975 non-null  object 
 2   Date                128975 non-null  object 
 3   Status              128975 non-null  object 
 4   Fulfilment          128975 non-null  object 
 5   Sales Channe

In [ ]:
# Assume your DataFrame is named Amazon_Report
# Convert the 'Date' column to datetime format
Amazon_Report["DATE_parsed"] = pd.to_datetime(
    Amazon_Report["Date"].astype("string").str.strip(),
    format="%m-%d-%y",
    errors="coerce"  # invalid formats will become NaT
)

# Create new date-related columns
Amazon_Report["day"]     = Amazon_Report["DATE_parsed"].dt.day
Amazon_Report["month"]   = Amazon_Report["DATE_parsed"].dt.month
Amazon_Report["year"]    = Amazon_Report["DATE_parsed"].dt.year
Amazon_Report["weekday"] = Amazon_Report["DATE_parsed"].dt.weekday   # 0 = Monday, 6 = Sunday
Amazon_Report["weekend"] = Amazon_Report["DATE_parsed"].dt.weekday >= 5  # True for Saturday/Sunday

# Convert data types safely (handles NaT values)
Amazon_Report["day"]     = Amazon_Report["day"].astype("Int64")
Amazon_Report["month"]   = Amazon_Report["month"].astype("Int64")
Amazon_Report["year"]    = Amazon_Report["year"].astype("Int64")
Amazon_Report["weekday"] = Amazon_Report["weekday"].astype("Int64")
Amazon_Report["weekend"] = Amazon_Report["weekend"].astype("boolean")

# Preview the first rows to verify changes
print(Amazon_Report.head())


   index             Order ID      Date                        Status  \
0      0  405-8078784-5731545  04-30-22                     Cancelled   
1      1  171-9198151-1101146  04-30-22  Shipped - Delivered to Buyer   
2      2  404-0687676-7273146  04-30-22                       Shipped   
3      3  403-9615377-8133951  04-30-22                     Cancelled   
4      4  407-1069790-7240320  04-30-22                       Shipped   

  Fulfilment Sales Channel  ship-service-level    Style              SKU  \
0   Merchant      Amazon.in           Standard   SET389   SET389-KR-NP-S   
1   Merchant      Amazon.in           Standard  JNE3781  JNE3781-KR-XXXL   
2     Amazon      Amazon.in          Expedited  JNE3371    JNE3371-KR-XL   
3   Merchant      Amazon.in           Standard    J0341       J0341-DR-L   
4     Amazon      Amazon.in          Expedited  JNE3671  JNE3671-TU-XXXL   

        Category  ...   SKU_ID SKU_SIZE   SKU_ID_SIZE        SKU_clean  \
0            Set  ...   SET389

In [ ]:
Amazon_Report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 34 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   index               128975 non-null  int64         
 1   Order ID            128975 non-null  object        
 2   Date                128975 non-null  object        
 3   Status              128975 non-null  object        
 4   Fulfilment          128975 non-null  object        
 5   Sales Channel       128975 non-null  object        
 6   ship-service-level  128975 non-null  object        
 7   Style               128975 non-null  object        
 8   SKU                 128975 non-null  object        
 9   Category            128975 non-null  object        
 10  Size                128975 non-null  object        
 11  ASIN                128975 non-null  object        
 12  Courier Status      122103 non-null  object        
 13  Qty                 128975 no

In [ ]:
Cloud  = pd.read_csv(r"Cloud Warehouse Compersion Chart.csv")
Cloud.info()
Cloud.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   index       50 non-null     int64 
 1   Shiprocket  21 non-null     object
 2   Unnamed: 1  41 non-null     object
 3   INCREFF     28 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.7+ KB


,index,Shiprocket,Unnamed: 1,INCREFF
0,0,Heads,Price (Per Unit),Price (Per Unit)
1,1,Inbound (Fresh Stock and RTO),₹4.00,4
2,2,Outbound,₹7.00,11
3,3,Storage Fee/Cft,₹25.00,Rs 0.15/- Per Day
4,4,Customer Return with Detailed QC,₹6.00,15.5


In [ ]:
Expense  = pd.read_csv(r"Expense IIGF.csv")
Expense.info()
Expense.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   index           17 non-null     int64 
 1   Recived Amount  6 non-null      object
 2   Unnamed: 1      6 non-null      object
 3   Expance         15 non-null     object
 4   Unnamed: 3      17 non-null     object
dtypes: int64(1), object(4)
memory usage: 812.0+ bytes


,index,Recived Amount,Unnamed: 1,Expance,Unnamed: 3
0,0,Particular,Amount,Particular,Amount
1,1,06-19-22,1000,Large Bag,380
2,2,06-20-22,1500,"Stationary(Soft Pin, Paper pin for Dupatta, Fe...",170
3,3,06-22-22,500,OLA,839
4,4,06-23-22,2000,Auto Rent,520


In [ ]:
International_sale  = pd.read_csv(r"International sale Report.csv")
International_sale.info()
International_sale.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37432 entries, 0 to 37431
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      37432 non-null  int64 
 1   DATE       37431 non-null  object
 2   Months     37407 non-null  object
 3   CUSTOMER   36392 non-null  object
 4   Style      36392 non-null  object
 5   SKU        34958 non-null  object
 6   Size       36392 non-null  object
 7   PCS        36392 non-null  object
 8   RATE       36392 non-null  object
 9   GROSS AMT  36392 non-null  object
dtypes: int64(1), object(9)
memory usage: 2.9+ MB


,index,DATE,Months,CUSTOMER,Style,SKU,Size,PCS,RATE,GROSS AMT
0,0,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-L,L,1.00,616.56,617.00
1,1,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-XL,XL,1.00,616.56,617.00
2,2,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-XXL,XXL,1.00,616.56,617.00
3,3,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5009,MEN5009-KR-L,L,1.00,616.56,617.00
4,4,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5011,MEN5011-KR-L,L,1.00,616.56,617.00


In [ ]:
# === 1) Parse with the exact formats you specified ===
date_parsed = pd.to_datetime(
    International_sale["DATE"].astype("string").str.strip(),
    format="%m-%d-%y",
    errors="coerce"
)

month_dt = pd.to_datetime(
    International_sale["Months"].astype("string").str.strip(),
    format="%b-%y",
    errors="coerce"
)
month_period = month_dt.dt.to_period("M")

# === 2) Build filters ===
start_date = pd.to_datetime("2022-03-31")
end_date   = pd.to_datetime("2022-06-29")

# If DATE is valid: filter by the inclusive date range
mask_by_date = date_parsed.between(start_date, end_date, inclusive="both")

# If DATE is missing: fallback by month—keep only April/May/June 2022
allowed_months = pd.PeriodIndex(["2022-04", "2022-05", "2022-06"], freq="M")
mask_by_month  = date_parsed.isna() & month_period.isin(allowed_months)

mask_keep = mask_by_date | mask_by_month

# Filter the table in place (use copy to avoid SettingWithCopy warnings)
International_sale = International_sale.loc[mask_keep].copy()

# === 3) Add 5 new columns ===
# Align the parsed series to the filtered index
date_parsed_f = date_parsed.reindex(International_sale.index)
month_dt_f    = month_dt.reindex(International_sale.index)

# Initialize new columns
International_sale["day"]     = pd.NA
International_sale["month"]   = pd.NA
International_sale["year"]    = pd.NA
International_sale["weekday"] = pd.NA
International_sale["weekend"] = pd.NA

# Rows with a full DATE: derive all fields from the parsed date
mask_has_date = date_parsed_f.notna()
International_sale.loc[mask_has_date, "day"]     = date_parsed_f.loc[mask_has_date].dt.day
International_sale.loc[mask_has_date, "month"]   = date_parsed_f.loc[mask_has_date].dt.month
International_sale.loc[mask_has_date, "year"]    = date_parsed_f.loc[mask_has_date].dt.year
# International_sale.loc[mask_has_date, "weekday"] = date_parsed_f.loc[mask_has_date].dt.day_name()
International_sale.loc[mask_has_date, "weekend"] = date_parsed_f.loc[mask_has_date].dt.weekday >= 5  # Sat/Sun
International_sale.loc[mask_has_date, "weekday"] = date_parsed_f.loc[mask_has_date].dt.weekday

# Rows without DATE but with Month: fill only month/year (day/weekday/weekend remain NA)
mask_only_month = (~mask_has_date) & month_dt_f.notna()
International_sale.loc[mask_only_month, "month"] = month_dt_f.loc[mask_only_month].dt.month
International_sale.loc[mask_only_month, "year"]  = month_dt_f.loc[mask_only_month].dt.year

# Set column dtypes
for col in ["day", "month", "year"]:
    International_sale[col] = pd.to_numeric(International_sale[col], errors="coerce").astype("Int64")
International_sale["weekend"] = International_sale["weekend"].astype("boolean")

# Quick sanity check
print(
    f"Rows kept: {len(International_sale)} | range: {start_date.date()}–{end_date.date()}"
)
print(International_sale[["DATE","Months","day","month","year","weekday","weekend"]].tail())


Rows kept: 1423 | range: 2022-03-31–2022-06-29
           DATE  Months  day  month  year weekday  weekend
18630  04-30-22  Apr-22   30      4  2022       5     True
18631  04-30-22  Apr-22   30      4  2022       5     True
18632  04-30-22  Apr-22   30      4  2022       5     True
18633  04-30-22  Apr-22   30      4  2022       5     True
18634  04-30-22  Apr-22   30      4  2022       5     True


In [ ]:
International_sale.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1423 entries, 17212 to 18634
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   index      1423 non-null   int64  
 1   DATE       1423 non-null   object 
 2   Months     1423 non-null   object 
 3   CUSTOMER   1423 non-null   object 
 4   Style      1423 non-null   object 
 5   SKU        23 non-null     object 
 6   Size       1423 non-null   object 
 7   PCS        1423 non-null   object 
 8   RATE       1423 non-null   object 
 9   GROSS AMT  1423 non-null   object 
 10  day        1423 non-null   Int64  
 11  month      1423 non-null   Int64  
 12  year       1423 non-null   Int64  
 13  weekday    1423 non-null   object 
 14  weekend    1423 non-null   boolean
dtypes: Int64(3), boolean(1), int64(1), object(10)
memory usage: 173.7+ KB


In [ ]:
International_sale['DATE'].value_counts()#lots of errors info

International_sale['DATE_clean'] = pd.to_datetime(
    International_sale['DATE'],
    format='%m-%d-%y',
    errors='coerce'
)
International_sale['DATE_clean'].value_counts()
mask_not_dates = International_sale['DATE_clean'].isna()
# Build candidate values from DATE (non-dates only)
vals = (
    International_sale
    .loc[mask_not_dates, 'DATE']
    .astype('string')
    .str.strip()
)

# SKU pattern: letters + digits (ignore spaces); allow _, -, /
sku_mask = (
    vals.str.replace(r"\s+", "", regex=True)
        .str.fullmatch(r"(?=.*[A-Za-z])(?=.*\d)[A-Za-z0-9_\-\/]+")
        .fillna(False)
)

idx_sku = vals.index[sku_mask]
mask_idx_sku = International_sale.index.isin(idx_sku)

# Save to SKU_from_DATE only where SKU is empty
International_sale.loc[
    mask_idx_sku & International_sale['SKU'].isna(),
    'SKU_from_DATE'
] = International_sale.loc[mask_idx_sku, 'DATE']

# Also fill original SKU where empty
International_sale.loc[
    mask_idx_sku & International_sale['SKU'].isna(),
    'SKU'
] = International_sale.loc[mask_idx_sku, 'DATE']
International_sale.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1423 entries, 17212 to 18634
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          1423 non-null   int64         
 1   DATE           1423 non-null   object        
 2   Months         1423 non-null   object        
 3   CUSTOMER       1423 non-null   object        
 4   Style          1423 non-null   object        
 5   SKU            23 non-null     object        
 6   Size           1423 non-null   object        
 7   PCS            1423 non-null   object        
 8   RATE           1423 non-null   object        
 9   GROSS AMT      1423 non-null   object        
 10  day            1423 non-null   Int64         
 11  month          1423 non-null   Int64         
 12  year           1423 non-null   Int64         
 13  weekday        1423 non-null   object        
 14  weekend        1423 non-null   boolean       
 15  DATE_clean     1423 n

In [ ]:
import pandas as pd
import numpy as np

# Assume your DataFrame is named International_sale
# Create clean string versions and remove extra spaces
sku   = International_sale["SKU"].astype("string").str.strip()
style = International_sale["Style"].astype("string").str.strip()
size  = International_sale["Size"].astype("string").str.strip()

# Create a candidate SKU based on "Style-Size" pattern
# Clean duplicate hyphens and trim leading/trailing hyphens
candidate = (style.fillna("") + "-" + size.fillna(""))
candidate = (candidate
             .str.replace(r"-{2,}", "-", regex=True)  # merge multiple consecutive hyphens
             .str.strip("-")                          # remove leading/trailing hyphens
            )

# If both Style and Size are missing → replace with NaN
candidate = candidate.replace("", pd.NA)

# Build the new SKU column:
# Keep existing SKUs; fill missing ones with the generated "Style-Size" candidates
International_sale["SKU_new"] = sku.copy()
mask = sku.isna()
International_sale.loc[mask, "SKU_new"] = candidate[mask]

# Optional: enforce uppercase format for consistency
# International_sale["SKU_new"] = International_sale["SKU_new"].str.upper()


In [ ]:
International_sale

,index,DATE,Months,CUSTOMER,Style,SKU,Size,PCS,RATE,GROSS AMT,day,month,year,weekday,weekend,DATE_clean,SKU_from_DATE,SKU_new
17212,17212,03-31-22,Mar-22,AVIN,JNE3421,JNE3421-KR-XL,XL,1.00,362.50,362.00,31,3,2022,3,False,2022-03-31,NaN,JNE3421-KR-XL
17213,17213,03-31-22,Mar-22,AVIN,JNE3672,JNE3672-TU-XL,XL,1.00,500.00,500.00,31,3,2022,3,False,2022-03-31,NaN,JNE3672-TU-XL
17214,17214,03-31-22,Mar-22,AVIN,SET272,SET272-KR-PP-XL,XL,1.00,887.50,888.00,31,3,2022,3,False,2022-03-31,NaN,SET272-KR-PP-XL
17215,17215,03-31-22,Mar-22,AVIN,SET203,SET203-KR-DPT-XS,XS,1.00,531.25,531.00,31,3,2022,3,False,2022-03-31,NaN,SET203-KR-DPT-XS
17216,17216,03-31-22,Mar-22,AVIN,J0213,J0213-TP-M,M,1.00,537.50,538.00,31,3,2022,3,False,2022-03-31,NaN,J0213-TP-M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18630,18630,04-30-22,Apr-22,Rino Sandaran,J0280,NaN,S,1.00,1812.50,1812.50,30,4,2022,5,True,2022-04-30,NaN,J0280-S
18631,18631,04-30-22,Apr-22,Rino Sandaran,SET350,NaN,M,1.00,1512.50,1512.50,30,4,2022,5,True,2022-04-30,NaN,SET350-M
18632,18632,04-30-22,Apr-22,Rino Sandaran,SET350,NaN,L,1.00,1512.50,1512.50,30,4,2022,5,True,2022-04-30,NaN,SET350-L
18633,18633,04-30-22,Apr-22,Rino Sandaran,J0242,NaN,XXXL,1.00,1312.50,1312.50,30,4,2022,5,True,2022-04-30,NaN,J0242-XXXL


In [ ]:
# Display all unique SKU values
International_sale.SKU.unique()

# Extract the first part before the first hyphen ('-')
International_sale["SKU_ID"] = International_sale["SKU_new"].str.split("-", n=1).str[0]

# Extract the last part after the last hyphen ('-')
International_sale["SKU_SIZE"] = International_sale["SKU_new"].str.rsplit("-", n=1).str[-1]

# Create a combined identifier column using SKU_ID and SKU_SIZE
International_sale["SKU_ID_SIZE"] = International_sale["SKU_ID"] + "-" + International_sale["SKU_SIZE"]

# Print the updated DataFrame to verify results
print(International_sale)

# Display summary information about the DataFrame (columns, data types, memory usage, etc.)
International_sale.info()


       index      DATE  Months       CUSTOMER    Style               SKU  \
17212  17212  03-31-22  Mar-22         AVIN    JNE3421     JNE3421-KR-XL   
17213  17213  03-31-22  Mar-22         AVIN    JNE3672     JNE3672-TU-XL   
17214  17214  03-31-22  Mar-22         AVIN     SET272   SET272-KR-PP-XL   
17215  17215  03-31-22  Mar-22         AVIN     SET203  SET203-KR-DPT-XS   
17216  17216  03-31-22  Mar-22         AVIN      J0213        J0213-TP-M   
...      ...       ...     ...            ...      ...               ...   
18630  18630  04-30-22  Apr-22  Rino Sandaran    J0280               NaN   
18631  18631  04-30-22  Apr-22  Rino Sandaran   SET350               NaN   
18632  18632  04-30-22  Apr-22  Rino Sandaran   SET350               NaN   
18633  18633  04-30-22  Apr-22  Rino Sandaran    J0242               NaN   
18634  18634  04-30-22  Apr-22  Rino Sandaran   SET043               NaN   

       Size   PCS     RATE GROSS AMT  ...  month  year  weekday weekend  \
17212    XL 

In [ ]:
International_sale

,index,DATE,Months,CUSTOMER,Style,SKU,Size,PCS,RATE,GROSS AMT,...,month,year,weekday,weekend,DATE_clean,SKU_from_DATE,SKU_new,SKU_ID,SKU_SIZE,SKU_ID_SIZE
17212,17212,03-31-22,Mar-22,AVIN,JNE3421,JNE3421-KR-XL,XL,1.00,362.50,362.00,...,3,2022,3,False,2022-03-31,NaN,JNE3421-KR-XL,JNE3421,XL,JNE3421-XL
17213,17213,03-31-22,Mar-22,AVIN,JNE3672,JNE3672-TU-XL,XL,1.00,500.00,500.00,...,3,2022,3,False,2022-03-31,NaN,JNE3672-TU-XL,JNE3672,XL,JNE3672-XL
17214,17214,03-31-22,Mar-22,AVIN,SET272,SET272-KR-PP-XL,XL,1.00,887.50,888.00,...,3,2022,3,False,2022-03-31,NaN,SET272-KR-PP-XL,SET272,XL,SET272-XL
17215,17215,03-31-22,Mar-22,AVIN,SET203,SET203-KR-DPT-XS,XS,1.00,531.25,531.00,...,3,2022,3,False,2022-03-31,NaN,SET203-KR-DPT-XS,SET203,XS,SET203-XS
17216,17216,03-31-22,Mar-22,AVIN,J0213,J0213-TP-M,M,1.00,537.50,538.00,...,3,2022,3,False,2022-03-31,NaN,J0213-TP-M,J0213,M,J0213-M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18630,18630,04-30-22,Apr-22,Rino Sandaran,J0280,NaN,S,1.00,1812.50,1812.50,...,4,2022,5,True,2022-04-30,NaN,J0280-S,J0280,S,J0280-S
18631,18631,04-30-22,Apr-22,Rino Sandaran,SET350,NaN,M,1.00,1512.50,1512.50,...,4,2022,5,True,2022-04-30,NaN,SET350-M,SET350,M,SET350-M
18632,18632,04-30-22,Apr-22,Rino Sandaran,SET350,NaN,L,1.00,1512.50,1512.50,...,4,2022,5,True,2022-04-30,NaN,SET350-L,SET350,L,SET350-L
18633,18633,04-30-22,Apr-22,Rino Sandaran,J0242,NaN,XXXL,1.00,1312.50,1312.50,...,4,2022,5,True,2022-04-30,NaN,J0242-XXXL,J0242,XXXL,J0242-XXXL


In [ ]:
# 1) Base: keep valid SKUs (must contain letters + digits) and convert to UPPER
s = International_sale["SKU"].astype("string")
mask_valid = s.str.contains(r"(?i)(?=.*[A-Za-z])(?=.*\d)", na=False)
International_sale["SKU_clean"] = s.where(mask_valid).str.upper()

# 2) Fallback without manipulation: build STYLE-SIZE and convert to UPPER
style_raw = International_sale.get("Style", pd.Series(pd.NA, index=International_sale.index)).astype("string")
size_raw  = International_sale.get("Size",  pd.Series(pd.NA, index=International_sale.index)).astype("string")
fallback  = (style_raw + "-" + size_raw).str.upper()

# Enforce consistency: the fallback must also include both letters and digits
fallback_valid = fallback.str.contains(r"(?=.*[A-Za-z])(?=.*\d)", na=False)

fill_mask = International_sale["SKU_clean"].isna() & fallback_valid
International_sale.loc[fill_mask, "SKU_clean"] = fallback.loc[fill_mask]

# Short report
print("Kept original valid SKU     :", int(mask_valid.sum()))
print("Filled from Style-Size      :", int(fill_mask.sum()))
print("Remaining NaN in SKU_clean  :", int(International_sale["SKU_clean"].isna().sum()))
print(International_sale.loc[fill_mask, ["SKU","Style","Size","SKU_clean"]].head(10))


Kept original valid SKU     : 23
Filled from Style-Size      : 1400
Remaining NaN in SKU_clean  : 0
       SKU   Style Size  SKU_clean
17235  NaN  SET322    L   SET322-L
17236  NaN  SET322   XL  SET322-XL
17237  NaN  SET197   XL  SET197-XL
17238  NaN  SET197    L   SET197-L
17239  NaN  SET197    M   SET197-M
17240  NaN  SET320   XL  SET320-XL
17241  NaN  SET320    L   SET320-L
17242  NaN  SET320    M   SET320-M
17243  NaN   J0279    M    J0279-M
17244  NaN   J0280    M    J0280-M


In [ ]:
# Unique Year–Month as strings: ['2021-06', '2021-07', ...]
unique_year_month = (
    International_sale['DATE_clean']
    .dropna()
    .dt.to_period('M')       # collapse to Month period
    .drop_duplicates()
    .sort_values()
    .astype(str)
    .tolist()
)
print(unique_year_month)


['2022-03', '2022-04', '2022-05']


In [ ]:
May  = pd.read_csv(r"May-2022.csv")
# הוספת שתי עמודות קבועות
# May["month"] = 5
# May["year"] = 2022

print(May.head())

May.info()
May.head()

   index             Sku    Style Id  Catalog Category Weight   TP MRP Old  \
0      0    Os206_3141_S  Os206_3141  Moments    Kurta    0.3  538    2178   
1      1    Os206_3141_M  Os206_3141  Moments    Kurta    0.3  538    2178   
2      2    Os206_3141_L  Os206_3141  Moments    Kurta    0.3  538    2178   
3      3   Os206_3141_XL  Os206_3141  Moments    Kurta    0.3  538    2178   
4      4  Os206_3141_2XL  Os206_3141  Moments    Kurta    0.3  538    2178   

  Final MRP Old Ajio MRP Amazon MRP Amazon FBA MRP Flipkart MRP Limeroad MRP  \
0          2295     2295       2295           2295         2295         2295   
1          2295     2295       2295           2295         2295         2295   
2          2295     2295       2295           2295         2295         2295   
3          2295     2295       2295           2295         2295         2295   
4          2295     2295       2295           2295         2295         2295   

  Myntra MRP Paytm MRP Snapdeal MRP  
0       2295

,index,Sku,Style Id,Catalog,Category,Weight,TP,MRP Old,Final MRP Old,Ajio MRP,Amazon MRP,Amazon FBA MRP,Flipkart MRP,Limeroad MRP,Myntra MRP,Paytm MRP,Snapdeal MRP
0,0,Os206_3141_S,Os206_3141,Moments,Kurta,0.3,538,2178,2295,2295,2295,2295,2295,2295,2295,2295,2295
1,1,Os206_3141_M,Os206_3141,Moments,Kurta,0.3,538,2178,2295,2295,2295,2295,2295,2295,2295,2295,2295
2,2,Os206_3141_L,Os206_3141,Moments,Kurta,0.3,538,2178,2295,2295,2295,2295,2295,2295,2295,2295,2295
3,3,Os206_3141_XL,Os206_3141,Moments,Kurta,0.3,538,2178,2295,2295,2295,2295,2295,2295,2295,2295,2295
4,4,Os206_3141_2XL,Os206_3141,Moments,Kurta,0.3,538,2178,2295,2295,2295,2295,2295,2295,2295,2295,2295


In [ ]:
import pandas as pd
import numpy as np

# Make a copy of the May DataFrame to avoid modifying the original
May = May.copy()

# Rename column 'TP' to 'TP1'
May = May.rename(columns={"TP": "TP1"})

# Add a new column 'TP2' initialized with NaN values
May["TP2"] = np.nan

# Display the first few rows of TP1 and TP2 for verification
print(May[["TP1", "TP2"]].head())


   TP1  TP2
0  538  NaN
1  538  NaN
2  538  NaN
3  538  NaN
4  538  NaN


In [ ]:
May.Sku.unique()

array(['Os206_3141_S', 'Os206_3141_M', 'Os206_3141_L', ..., 'Os326_XL',
       'Os326_2XL', 'Os326_3XL'], dtype=object)

In [ ]:
# Display SKU value counts to understand frequency distribution
May.Sku.value_counts()

# Create a cleaned SKU column in uppercase for consistency
May["SKU_clean"] = May["Sku"].astype("string").str.upper()

# Extract the first part before the first underscore ("_")
May["SKU_ID"] = May["SKU_clean"].str.split("_", n=1).str[0]

# Extract the last part after the last underscore ("_")
May["SKU_SIZE"] = May["SKU_clean"].str.rsplit("_", n=1).str[-1]

# Create a combined identifier SKU_ID_SIZE with a hyphen
May["SKU_ID_SIZE"] = May["SKU_ID"] + "-" + May["SKU_SIZE"]

# Print the updated DataFrame for verification
print(May)

# Display structural info about the DataFrame (column names, data types, memory usage, etc.)
May.info()


      index             Sku    Style Id  Catalog Category Weight  TP1 MRP Old  \
0         0    Os206_3141_S  Os206_3141  Moments    Kurta    0.3  538    2178   
1         1    Os206_3141_M  Os206_3141  Moments    Kurta    0.3  538    2178   
2         2    Os206_3141_L  Os206_3141  Moments    Kurta    0.3  538    2178   
3         3   Os206_3141_XL  Os206_3141  Moments    Kurta    0.3  538    2178   
4         4  Os206_3141_2XL  Os206_3141  Moments    Kurta    0.3  538    2178   
...     ...             ...         ...      ...      ...    ...  ...     ...   
1325   1325         Os326_M       Os326      Mix    Kurta    0.3  467    1878   
1326   1326         Os326_L       Os326      Mix    Kurta    0.3  467    1878   
1327   1327        Os326_XL       Os326      Mix    Kurta    0.3  467    1878   
1328   1328       Os326_2XL       Os326      Mix    Kurta    0.3  467    1878   
1329   1329       Os326_3XL       Os326      Mix    Kurta    0.3  467    1878   

     Final MRP Old Ajio MRP

In [ ]:
March  = pd.read_csv(r"P  L March 2021.csv")
# March['month']=3
# March['year']=2022
March.info()
March.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1330 entries, 0 to 1329
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   index           1330 non-null   int64 
 1   Sku             1330 non-null   object
 2   Style Id        1330 non-null   object
 3   Catalog         1330 non-null   object
 4   Category        1330 non-null   object
 5   Weight          1330 non-null   object
 6   TP 1            1330 non-null   object
 7   TP 2            1330 non-null   object
 8   MRP Old         1330 non-null   object
 9   Final MRP Old   1330 non-null   object
 10  Ajio MRP        1330 non-null   object
 11  Amazon MRP      1330 non-null   object
 12  Amazon FBA MRP  1330 non-null   object
 13  Flipkart MRP    1330 non-null   object
 14  Limeroad MRP    1330 non-null   object
 15  Myntra MRP      1330 non-null   object
 16  Paytm MRP       1330 non-null   object
 17  Snapdeal MRP    1330 non-null   object
dtypes: int64

,index,Sku,Style Id,Catalog,Category,Weight,TP 1,TP 2,MRP Old,Final MRP Old,Ajio MRP,Amazon MRP,Amazon FBA MRP,Flipkart MRP,Limeroad MRP,Myntra MRP,Paytm MRP,Snapdeal MRP
0,0,Os206_3141_S,Os206_3141,Moments,Kurta,0.3,538,435.78,2178,2295,2295,2295,2295,2295,2295,2295,2295,2295
1,1,Os206_3141_M,Os206_3141,Moments,Kurta,0.3,538,435.78,2178,2295,2295,2295,2295,2295,2295,2295,2295,2295
2,2,Os206_3141_L,Os206_3141,Moments,Kurta,0.3,538,435.78,2178,2295,2295,2295,2295,2295,2295,2295,2295,2295
3,3,Os206_3141_XL,Os206_3141,Moments,Kurta,0.3,538,435.78,2178,2295,2295,2295,2295,2295,2295,2295,2295,2295
4,4,Os206_3141_2XL,Os206_3141,Moments,Kurta,0.3,538,435.78,2178,2295,2295,2295,2295,2295,2295,2295,2295,2295


In [ ]:
# Display SKU value counts to inspect frequency distribution
March.Sku.value_counts()

# Create a cleaned SKU column by converting to uppercase for consistency
March["SKU_clean"] = March["Sku"].astype("string").str.upper()

# Extract the first part before the first underscore ("_")
March["SKU_ID"] = March["SKU_clean"].str.split("_", n=1).str[0]

# Extract the last part after the last underscore ("_")
March["SKU_SIZE"] = March["SKU_clean"].str.rsplit("_", n=1).str[-1]

# Combine both parts into a single column SKU_ID_SIZE with a hyphen separator
March["SKU_ID_SIZE"] = March["SKU_ID"] + "-" + March["SKU_SIZE"]

# Print the updated DataFrame for verification
print(March)

# Display summary information (columns, data types, non-null counts, memory usage, etc.)
March.info()


      index             Sku    Style Id  Catalog Category Weight TP 1    TP 2  \
0         0    Os206_3141_S  Os206_3141  Moments    Kurta    0.3  538  435.78   
1         1    Os206_3141_M  Os206_3141  Moments    Kurta    0.3  538  435.78   
2         2    Os206_3141_L  Os206_3141  Moments    Kurta    0.3  538  435.78   
3         3   Os206_3141_XL  Os206_3141  Moments    Kurta    0.3  538  435.78   
4         4  Os206_3141_2XL  Os206_3141  Moments    Kurta    0.3  538  435.78   
...     ...             ...         ...      ...      ...    ...  ...     ...   
1325   1325         Os326_M       Os326      Mix    Kurta    0.3  467  378.27   
1326   1326         Os326_L       Os326      Mix    Kurta    0.3  467  378.27   
1327   1327        Os326_XL       Os326      Mix    Kurta    0.3  467  378.27   
1328   1328       Os326_2XL       Os326      Mix    Kurta    0.3  467  378.27   
1329   1329       Os326_3XL       Os326      Mix    Kurta    0.3  467  378.27   

     MRP Old Final MRP Old 

In [ ]:
Sale_Report  = pd.read_csv(r"Sale Report.csv")
Sale_Report.info()
Sale_Report.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9271 entries, 0 to 9270
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       9271 non-null   int64  
 1   SKU Code    9188 non-null   object 
 2   Design No.  9235 non-null   object 
 3   Stock       9235 non-null   float64
 4   Category    9226 non-null   object 
 5   Size        9235 non-null   object 
 6   Color       9226 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 507.1+ KB


,index,SKU Code,Design No.,Stock,Category,Size,Color
0,0,AN201-RED-L,AN201,5.0,AN : LEGGINGS,L,Red
1,1,AN201-RED-M,AN201,5.0,AN : LEGGINGS,M,Red
2,2,AN201-RED-S,AN201,3.0,AN : LEGGINGS,S,Red
3,3,AN201-RED-XL,AN201,6.0,AN : LEGGINGS,XL,Red
4,4,AN201-RED-XXL,AN201,3.0,AN : LEGGINGS,XXL,Red


In [ ]:
import pandas as pd
import numpy as np

# === Base: Clean and split 'SKU Code' ===
sku = Sale_Report["SKU Code"].astype("string").str.strip()

# Extract first part before the first hyphen
sku_id = sku.str.split("-", n=1).str[0]

# Extract last part after the last hyphen
sku_size = sku.str.rsplit("-", n=1).str[-1]

# Combine both parts into a full SKU identifier
sku_id_size = sku_id + "-" + sku_size

# === Fallback when 'SKU Code' is missing ===
design = Sale_Report["Design No."].astype("string").str.strip()
size   = Sale_Report["Size"].astype("string").str.strip()

sku_id_fallback = design
sku_size_fallback = size
sku_id_size_fallback = design + "-" + size

# === Build final columns ===
Sale_Report["SKU_ID"] = np.where(sku.notna(), sku_id, sku_id_fallback)
Sale_Report["SKU_SIZE"] = np.where(sku.notna(), sku_size, sku_size_fallback)
Sale_Report["SKU_ID_SIZE"] = np.where(sku.notna(), sku_id_size, sku_id_size_fallback)

# Display the updated DataFrame for verification
print(Sale_Report)

# Show detailed information about the DataFrame (columns, dtypes, memory usage, etc.)
Sale_Report.info()


      index       SKU Code Design No.  Stock       Category Size Color SKU_ID  \
0         0    AN201-RED-L      AN201    5.0  AN : LEGGINGS    L   Red  AN201   
1         1    AN201-RED-M      AN201    5.0  AN : LEGGINGS    M   Red  AN201   
2         2    AN201-RED-S      AN201    3.0  AN : LEGGINGS    S   Red  AN201   
3         3   AN201-RED-XL      AN201    6.0  AN : LEGGINGS   XL   Red  AN201   
4         4  AN201-RED-XXL      AN201    3.0  AN : LEGGINGS  XXL   Red  AN201   
...     ...            ...        ...    ...            ...  ...   ...    ...   
9266   9266            NaN        NaN    NaN            NaN  NaN   NaN   <NA>   
9267   9267            NaN        NaN    NaN            NaN  NaN   NaN   <NA>   
9268   9268            NaN        NaN    NaN            NaN  NaN   NaN   <NA>   
9269   9269            NaN        NaN    NaN            NaN  NaN   NaN   <NA>   
9270   9270            NaN        NaN    NaN            NaN  NaN   NaN   <NA>   

     SKU_SIZE SKU_ID_SIZE  

In [ ]:
Sale_Report = Sale_Report[Sale_Report["SKU_ID"].notna()].copy()
Sale_Report.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9235 entries, 0 to 9234
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        9235 non-null   int64  
 1   SKU Code     9188 non-null   object 
 2   Design No.   9235 non-null   object 
 3   Stock        9235 non-null   float64
 4   Category     9226 non-null   object 
 5   Size         9235 non-null   object 
 6   Color        9226 non-null   object 
 7   SKU_ID       9235 non-null   object 
 8   SKU_SIZE     9235 non-null   object 
 9   SKU_ID_SIZE  9235 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 793.6+ KB


In [ ]:
Sale_Report.SKU_ID_SIZE.unique()

array(['AN201-L', 'AN201-M', 'AN201-S', ..., 'SET462-XS', 'SET462-XXL',
       'SET462-XXXL'], dtype=object)

2. Merging tables

In [ ]:
# Standardize column names for both DataFrames (Amazon_Report and International_sale)
Amazon_Report.columns = (
    Amazon_Report.columns
    .str.strip()              # Remove leading/trailing spaces
    .str.lower()              # Convert all letters to lowercase
    .str.replace(" ", "_")    # Replace spaces with underscores
)

International_sale.columns = (
    International_sale.columns
    .str.strip()              # Remove leading/trailing spaces
    .str.lower()              # Convert all letters to lowercase
    .str.replace(" ", "_")    # Replace spaces with underscores
)

# Combine both DataFrames into one unified dataset
fact_all = pd.concat([Amazon_Report, International_sale], ignore_index=True)


In [ ]:
# Display detailed information about the combined DataFrame
fact_all.info()

# Save the merged dataset to a CSV file
fact_all.to_csv("fact_all.csv", index=False, encoding="utf-8-sig")
# index=False → do not save the index column
# encoding="utf-8-sig" → ensures proper encoding for Excel and other systems


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130398 entries, 0 to 130397
Data columns (total 42 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   index               130398 non-null  int64         
 1   order_id            128975 non-null  object        
 2   date                130398 non-null  object        
 3   status              128975 non-null  object        
 4   fulfilment          128975 non-null  object        
 5   sales_channel       128975 non-null  object        
 6   ship-service-level  128975 non-null  object        
 7   style               130398 non-null  object        
 8   sku                 128998 non-null  object        
 9   category            128975 non-null  object        
 10  size                130398 non-null  object        
 11  asin                128975 non-null  object        
 12  courier_status      122103 non-null  object        
 13  qty                 128975 no

In [ ]:
Amazon_Inter  = pd.read_csv(r"fact_all.csv")
Amazon_Inter.info()

/tmp/ipython-input-3032261130.py:1: DtypeWarning: Columns (21,23,34,35,39,41) have mixed types. Specify dtype option on import or set low_memory=False.
  Amazon_Inter  = pd.read_csv(r"fact_all.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130398 entries, 0 to 130397
Data columns (total 42 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               130398 non-null  int64  
 1   order_id            128975 non-null  object 
 2   date                130398 non-null  object 
 3   status              128975 non-null  object 
 4   fulfilment          128975 non-null  object 
 5   sales_channel       128975 non-null  object 
 6   ship-service-level  128975 non-null  object 
 7   style               130398 non-null  object 
 8   sku                 128998 non-null  object 
 9   category            128975 non-null  object 
 10  size                130398 non-null  object 
 11  asin                128975 non-null  object 
 12  courier_status      122103 non-null  object 
 13  qty                 128975 non-null  float64
 14  currency            121180 non-null  object 
 15  amount              121180 non-nul

In [ ]:
# Standardize column names for March DataFrame
March.columns = (
    March.columns
    .str.strip()              # Remove leading/trailing spaces
    .str.lower()              # Convert all letters to lowercase
    .str.replace(" ", "_")    # Replace spaces with underscores
)

# Standardize column names for May DataFrame
May.columns = (
    May.columns
    .str.strip()              # Remove leading/trailing spaces
    .str.lower()              # Convert all letters to lowercase
    .str.replace(" ", "_")    # Replace spaces with underscores
)

# Standardize column names for Sale_Report DataFrame
Sale_Report.columns = (
    Sale_Report.columns
    .str.strip()              # Remove leading/trailing spaces
    .str.lower()              # Convert all letters to lowercase
    .str.replace(" ", "_")    # Replace spaces with underscores
)

# Display the updated column names of Sale_Report
Sale_Report.columns


Index(['index', 'sku_code', 'design_no.', 'stock', 'category', 'size', 'color',
       'sku_id', 'sku_size', 'sku_id_size'],
      dtype='object')

In [ ]:
# START HERE
# ==== 1) FACT: Merge all sales (Amazon + International) into a single DataFrame ====

fact_all = pd.concat([Amazon_Report, International_sale], ignore_index=True)

# Parse the 'date' column to datetime; invalid or malformed rows become NaT
fact_all["date"] = pd.to_datetime(
    fact_all["date"].astype("string").str.strip(),
    # format="%m-%d-%y",  # MM-DD-YY (left commented on purpose)
    errors="coerce"   # invalid records will turn into NaT
)

# ==== 2) Split by cutoff between March and May ====
cutoff = pd.Timestamp("2022-05-01")
mask_pre  = fact_all["date"] < cutoff    # rows before May 1, 2022 → join with March catalog
mask_post = fact_all["date"] >= cutoff   # rows from May 1, 2022 onward → join with May catalog

# Prepare catalog column lists (exclude index columns if present)
catalog_march_cols = [c for c in March.columns if c not in ["index"]]
catalog_may_cols   = [c for c in May.columns if c not in ["index"]]

# Merge pre-cutoff subset with March catalog on 'sku_id_size'
pre_part  = fact_all.loc[mask_pre].merge(March[catalog_march_cols],
                                         on="sku_id_size", how="left", suffixes=("","_march"))

# Merge post-cutoff subset with May catalog on 'sku_id_size'
post_part = fact_all.loc[mask_post].merge(May[catalog_may_cols],
                                         on="sku_id_size", how="left", suffixes=("","_may"))

# Recombine both subsets back into one table
fact_with_catalog = pd.concat([pre_part, post_part], ignore_index=True)

# ==== 3) Join stock data (Sale_Report) ====
# Prepare stock columns (exclude index columns if present)
sale_cols = [c for c in Sale_Report.columns if c not in ["index"]]

# Left-join stock info by 'sku_id_size' to preserve all sales rows
best_dataset = fact_with_catalog.merge(Sale_Report[sale_cols],
                                       on="sku_id_size", how="left", suffixes=("","_stock"))

# ==== 4) Quick checks ====
# Print row/column counts and original sources' column counts (Hebrew labels kept as-is intentionally)
print("Raws:", best_dataset.shape[0])
print("Columns:", best_dataset.shape[1])
print("Original columns number:", {
    "Amazon": len(Amazon_Report.columns),
    "International": len(International_sale.columns),
    "March": len(March.columns),
    "May": len(May.columns),
    "Sale_Report": len(Sale_Report.columns)
})


/tmp/ipython-input-2424506345.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  fact_all["date"] = pd.to_datetime(


שורות: 130426
עמודות: 77
מס׳ עמודות מקור: {'Amazon': 34, 'International': 22, 'March': 22, 'May': 22, 'Sale_Report': 10}


In [ ]:
best_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130426 entries, 0 to 130425
Data columns (total 77 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   index               130426 non-null  int64         
 1   order_id            129002 non-null  object        
 2   date                130426 non-null  datetime64[ns]
 3   status              129002 non-null  object        
 4   fulfilment          129002 non-null  object        
 5   sales_channel       129002 non-null  object        
 6   ship-service-level  129002 non-null  object        
 7   style               130426 non-null  object        
 8   sku                 129026 non-null  object        
 9   category            129002 non-null  object        
 10  size                130426 non-null  object        
 11  asin                129002 non-null  object        
 12  courier_status      122130 non-null  object        
 13  qty                 129002 no

In [ ]:
best_dataset.info()
best_dataset.to_csv("best_dataset.csv", index=False, encoding="utf-8-sig")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130426 entries, 0 to 130425
Data columns (total 77 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   index               130426 non-null  int64         
 1   order_id            129002 non-null  object        
 2   date                130426 non-null  datetime64[ns]
 3   status              129002 non-null  object        
 4   fulfilment          129002 non-null  object        
 5   sales_channel       129002 non-null  object        
 6   ship-service-level  129002 non-null  object        
 7   style               130426 non-null  object        
 8   sku                 129026 non-null  object        
 9   category            129002 non-null  object        
 10  size                130426 non-null  object        
 11  asin                129002 non-null  object        
 12  courier_status      122130 non-null  object        
 13  qty                 129002 no

In [ ]:
# Check if any SKU values are shared between May and Amazon_Report (overlap and coverage)

# === 1) Normalize keys (trim + upper) ===
may_key = (
    May["sku_id"].astype("string")
    .str.strip()
    .str.upper()
)  # normalize May SKU IDs

amz_key = (
    Amazon_Report["sku_id"].astype("string")
    .str.strip()
    .str.upper()
)  # normalize Amazon_Report SKU IDs

# === 2) Get unique sets to measure distinct overlap ===
may_uniq = pd.Index(may_key.dropna().unique())   # unique SKUs in May
amz_uniq = pd.Index(amz_key.dropna().unique())   # unique SKUs in Amazon_Report

overlap_uniq = may_uniq.intersection(amz_uniq)   # distinct SKUs present in both datasets

print("Distinct overlap (SKU_SIZE):", len(overlap_uniq))
print("Distinct in May:", len(may_uniq))
print("Distinct in Amazon_Report:", len(amz_uniq))
print("Distinct coverage of May in Amazon_Report: {:.2%}".format(len(overlap_uniq) / max(len(may_uniq), 1)))
# Coverage = what fraction of distinct May SKUs also appear in Amazon_Report

# === 3) Count row-level matches from May found in Amazon_Report (not distinct) ===
rows_in_may_found = may_key.isin(amz_uniq).sum()   # how many May rows have a SKU also in Amazon_Report
rows_in_may_total = may_key.shape[0]               # total May rows
print("Rows from May found in Amazon_Report:", rows_in_may_found, "/", rows_in_may_total,
      "({:.2%})".format(rows_in_may_found / max(rows_in_may_total, 1)))
# This shows row-level coverage, which can be higher than distinct coverage if SKUs repeat

# === 4) Sample May SKUs that DO NOT exist in Amazon_Report ===
missing_samples = May.loc[~may_key.isin(amz_uniq), ["sku_id"]].dropna().head(20)
print("\nSamples from May NOT in Amazon_Report:")
print(missing_samples.to_string(index=False))
# Use this sample to inspect formatting issues or genuine mismatches


Distinct overlap (SKU_SIZE): 0
Distinct in May: 250
Distinct in Amazon_Report: 1377
Distinct coverage of May in Amazon_Report: 0.00%
Rows from May found in Amazon_Report: 0 / 1330 (0.00%)

Samples from May NOT in Amazon_Report:
sku_id
 OS206
 OS206
 OS206
 OS206
 OS206
 OS206
 OS223
 OS223
 OS223
 OS223
 OS223
 OS223
 OS212
 OS212
 OS212
 OS212
 OS212
 OS212
 OS214
 OS214


In [ ]:
Amazon_Data  = pd.read_csv(r"best_dataset.csv")
Amazon_Data.info()

/tmp/ipython-input-1939351423.py:1: DtypeWarning: Columns (21,23,34,35,39,41) have mixed types. Specify dtype option on import or set low_memory=False.
  Amazon_Data  = pd.read_csv(r"best_dataset.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130426 entries, 0 to 130425
Data columns (total 77 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               130426 non-null  int64  
 1   order_id            129002 non-null  object 
 2   date                130426 non-null  object 
 3   status              129002 non-null  object 
 4   fulfilment          129002 non-null  object 
 5   sales_channel       129002 non-null  object 
 6   ship-service-level  129002 non-null  object 
 7   style               130426 non-null  object 
 8   sku                 129026 non-null  object 
 9   category            129002 non-null  object 
 10  size                130426 non-null  object 
 11  asin                129002 non-null  object 
 12  courier_status      122130 non-null  object 
 13  qty                 129002 non-null  float64
 14  currency            121204 non-null  object 
 15  amount              121204 non-nul

In [ ]:
# Identify columns that are entirely NaN (100% missing values)
all_null_cols = Amazon_Data.columns[Amazon_Data.isna().all()].tolist()
print("Columns with 100% NaN:", all_null_cols)

# Drop these fully empty columns from the DataFrame
Amazon_Data = Amazon_Data.drop(columns=all_null_cols)

# Print how many columns were removed and how many remain
print(f"Removed {len(all_null_cols)} columns. Remaining columns: {Amazon_Data.shape[1]}")

# Display the updated DataFrame structure (column names, data types, memory usage, etc.)
Amazon_Data.info()


עמודות 100% NaN: ['sku_from_date', 'sku_march', 'style_id', 'catalog', 'category_march', 'weight', 'tp_1', 'tp_2', 'mrp_old', 'final_mrp_old', 'ajio_mrp', 'amazon_mrp', 'amazon_fba_mrp', 'flipkart_mrp', 'limeroad_mrp', 'myntra_mrp', 'paytm_mrp', 'snapdeal_mrp', 'sku_clean_march', 'sku_id_march', 'sku_size_march', 'sku_may', 'category_may', 'tp1', 'tp2', 'sku_clean_may', 'sku_id_may', 'sku_size_may']
נמחקו 28 עמודות. נשארו 49 עמודות.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130426 entries, 0 to 130425
Data columns (total 49 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               130426 non-null  int64  
 1   order_id            129002 non-null  object 
 2   date                130426 non-null  object 
 3   status              129002 non-null  object 
 4   fulfilment          129002 non-null  object 
 5   sales_channel       129002 non-null  object 
 6   ship-service-level  129002 non-null  object 
 7

In [ ]:
import pandas as pd
import re, unicodedata as ud

DATE_COL = "date"  # Update if the column name differs

# --- Clean date text manually (without using .str) ---
def clean_date_text(x):
    if pd.isna(x):
        return pd.NA
    x = str(x)
    x = ud.normalize("NFKC", x)                      # Normalize Unicode
    x = re.sub(r"[\u00A0\u200B\u200E\u200F]", "", x) # Remove invisible characters
    x = x.strip()
    x = re.sub(r"\s+.*$", "", x)                     # Remove text after space (e.g., time)
    x = re.sub(r"[./\\_]", "-", x)                   # Unify separators to '-'
    x = re.sub(r"[–—−]+", "-", x)                    # Replace long dashes with '-'
    x = re.sub(r"-{2,}", "-", x).strip("-")          # Collapse repeated dashes and trim edges
    return x if x else pd.NA

# Apply cleaning function
s_raw   = Amazon_Data[DATE_COL]
s_clean = s_raw.map(clean_date_text)

# --- Try common date formats and choose the one that works best ---
candidates = [
    "%Y-%m-%d", "%m-%d-%Y", "%d-%m-%Y",
    "%m/%d/%Y", "%d/%m/%Y", "%Y/%m/%d",
    "%m-%d-%y", "%d-%m-%y",               # Two-digit years
    "%b-%d-%Y", "%d-%b-%Y", "%B-%d-%Y", "%d-%B-%Y"  # Month names
]

best_dt, best_fmt, best_ok = None, None, -1
for fmt in candidates:
    dt_try = pd.to_datetime(s_clean, format=fmt, errors="coerce")
    ok = int(dt_try.notna().sum())
    if ok > best_ok:
        best_dt, best_fmt, best_ok = dt_try, fmt, ok

# --- Fallback for numeric timestamps (Unix seconds/ms or Excel serial) ---
parsed = best_dt
if best_ok <= 0:
    nums = pd.to_numeric(s_clean, errors="coerce")
    dt_num = pd.Series(pd.NaT, index=nums.index, dtype="datetime64[ns]")
    # Unix milliseconds
    mask_ms = nums.between(10**11, 10**13)
    dt_num.loc[mask_ms] = pd.to_datetime(nums.loc[mask_ms], unit="ms", origin="unix")
    # Unix seconds
    mask_s = nums.between(10**8, 10**10)
    dt_num.loc[mask_s]  = pd.to_datetime(nums.loc[mask_s], unit="s", origin="unix")
    # Excel serial (days since 1899-12-30)
    mask_xl = nums.between(20000, 60000)
    if mask_xl.any():
        origin = pd.Timestamp("1899-12-30")
        dt_num.loc[mask_xl] = origin + nums.loc[mask_xl].astype(float).map(pd.Timedelta, unit="D")
    parsed   = dt_num
    best_fmt = "numeric (unix/excel)"

# Assign parsed date column back
Amazon_Data[DATE_COL] = parsed

# Report parsing results
print(f"parsed rows: {Amazon_Data[DATE_COL].notna().sum()} / {len(Amazon_Data)} using {best_fmt}")
if Amazon_Data[DATE_COL].isna().any():
    print("Problematic raw samples:", s_raw[Amazon_Data[DATE_COL].isna()].astype(str).head(10).to_list())

# --- Sort by date; NaT values placed last ---
Amazon_Data = Amazon_Data.sort_values(DATE_COL, ascending=True, na_position="last").reset_index(drop=True)


parsed rows: 130426 / 130426 using %Y-%m-%d


In [ ]:
Amazon_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130426 entries, 0 to 130425
Data columns (total 49 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   index               130426 non-null  int64         
 1   order_id            129002 non-null  object        
 2   date                130426 non-null  datetime64[ns]
 3   status              129002 non-null  object        
 4   fulfilment          129002 non-null  object        
 5   sales_channel       129002 non-null  object        
 6   ship-service-level  129002 non-null  object        
 7   style               130426 non-null  object        
 8   sku                 129026 non-null  object        
 9   category            129002 non-null  object        
 10  size                130426 non-null  object        
 11  asin                129002 non-null  object        
 12  courier_status      122130 non-null  object        
 13  qty                 129002 no

In [ ]:
import pandas as pd

# 3) (Optional) Daily transaction count
daily_counts = (
    Amazon_Data
    .set_index("date")   # Set the date column as index
    .assign(_n=1)        # Temporary counter column for counting rows
    .resample("D")["_n"] # Resample by day ("D" = daily frequency)
    .count()             # Count number of records per day
    .astype(int)         # Ensure integer type
    .rename("count")     # Rename series for clarity
)

# 4) (Optional) Quick validation
print("Date range:", Amazon_Data["date"].min(), "→", Amazon_Data["date"].max())
print(daily_counts.head())  # Show first few days
print(daily_counts.tail())  # Show last few days


date range: 2022-03-31 00:00:00 → 2022-06-29 00:00:00
date
2022-03-31     195
2022-04-01    1501
2022-04-02    1556
2022-04-03    1693
2022-04-04    1483
Freq: D, Name: count, dtype: int64
date
2022-06-25    1026
2022-06-26    1210
2022-06-27    1111
2022-06-28    1210
2022-06-29     658
Freq: D, Name: count, dtype: int64


In [ ]:
Amazon_Data.to_csv("best_dataset_clean_1.csv", index=False, encoding="utf-8-sig")

In [ ]:
import pandas as pd
Amazon_Data  = pd.read_csv(r"best_dataset_clean_1.csv")
Amazon_Data.info()

/tmp/ipython-input-2208241411.py:2: DtypeWarning: Columns (21,23,34,35,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  Amazon_Data  = pd.read_csv(r"best_dataset_clean_1.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130426 entries, 0 to 130425
Data columns (total 49 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               130426 non-null  int64  
 1   order_id            129002 non-null  object 
 2   date                130426 non-null  object 
 3   status              129002 non-null  object 
 4   fulfilment          129002 non-null  object 
 5   sales_channel       129002 non-null  object 
 6   ship-service-level  129002 non-null  object 
 7   style               130426 non-null  object 
 8   sku                 129026 non-null  object 
 9   category            129002 non-null  object 
 10  size                130426 non-null  object 
 11  asin                129002 non-null  object 
 12  courier_status      122130 non-null  object 
 13  qty                 129002 non-null  float64
 14  currency            121204 non-null  object 
 15  amount              121204 non-nul

In [ ]:
# Convert 'order_id' column to string type
Amazon_Data["order_id"] = Amazon_Data["order_id"].astype("string")
Amazon_Data["order_id"].dtype  # Confirm the new data type

# Review distribution of order statuses
Amazon_Data["status"].value_counts()

# Extract only the first word from each 'status' value
Amazon_Data["status_first_word"] = Amazon_Data["status"].str.split().str[0]

# Convert to categorical type for efficiency
Amazon_Data["status_first_word"] = Amazon_Data["status_first_word"].astype("category")

# Check frequency and data type of the new column
Amazon_Data["status_first_word"].value_counts()
Amazon_Data["status_first_word"].dtype

# Create a Boolean column: True if fulfilment is by Amazon
Amazon_Data["fulfilment_by_amazon"] = Amazon_Data["fulfilment"] == "Amazon"
Amazon_Data["fulfilment_by_amazon"].value_counts()

# Check the distribution of the sales channel
Amazon_Data["sales_channel"].value_counts()

# Create a Boolean flag for Amazon.in channel
Amazon_Data["amazon_channel"] = Amazon_Data["sales_channel"] == "Amazon.in"
Amazon_Data["amazon_channel"].value_counts()

# Check shipping service levels
Amazon_Data["ship-service-level"].value_counts()

# Convert 'ship-service-level' to an ordered categorical variable
Amazon_Data["ship-service-level"] = pd.Categorical(
    Amazon_Data["ship-service-level"],
    categories=["Standard", "Expedited"],   # logical order
    ordered=True
)

# Verify new category distribution
Amazon_Data["ship-service-level"].value_counts()

# Drop redundant columns after transformation
Amazon_Data = Amazon_Data.drop(columns=["status", "sales_channel"])


In [ ]:
# Check distribution of 'style' values
Amazon_Data["style"].value_counts()

# Copy 'sku_id' to a new column 'style_id' as string
Amazon_Data["style_id"] = Amazon_Data["sku_id"].astype("string")

# Ensure 'sku_id_size' is stored as string type
Amazon_Data["sku_id_size"] = Amazon_Data["sku_id_size"].astype("string")

# Review category distribution before normalization
Amazon_Data["category"].value_counts()

# Merge small categories: replace 'Dupatta' with 'Saree'
Amazon_Data["category"] = Amazon_Data["category"].replace(
    {"Dupatta": "Saree"}
)

# Convert 'category' to categorical type for efficiency
Amazon_Data["category"] = Amazon_Data["category"].astype("category")

# Review size distribution before normalization
Amazon_Data["size"].value_counts()

# Define normalization mapping for size values
size_map = {
    "m": "M",
    "l": "L",
    "s": "S",
    "xl": "XL",
    "xxl": "XXL",
    "xxxl": "3XL",
    "XXXL": "3XL",
    "S TO XXL": "Free",   # unified free size
    "6XL": "4XL",
    "5XL": "4XL"
}

# Apply size normalization
Amazon_Data["size"] = Amazon_Data["size"].replace(size_map)

# Convert normalized size column to categorical type
Amazon_Data["size"] = Amazon_Data["size"].astype("category")

# Verify SKU size distribution after cleaning
Amazon_Data["sku_size"].value_counts()

# Drop redundant columns no longer needed after processing
Amazon_Data = Amazon_Data.drop(columns=["style", "sku", "sku_size", "sku_id"])


In [ ]:
Amazon_Data["asin"] = Amazon_Data["asin"].astype("string")

Amazon_Data["courier_status"].value_counts()
Amazon_Data["courier_status"] = Amazon_Data["courier_status"].astype("category")

Amazon_Data["qty"].value_counts()
import numpy as np
def map_qty(val):
    if pd.isna(val):
        return np.nan
    elif val == 0.0:
        return "Zero"
    elif val == 1.0:
        return "One"
    elif val >= 2.0:
        return "TwoPlus"
    else:
        return np.nan

Amazon_Data["qty_cat"] = Amazon_Data["qty"].apply(map_qty)
cat_type = pd.CategoricalDtype(categories=["Zero", "One", "TwoPlus"], ordered=True)
Amazon_Data["qty_cat"] = Amazon_Data["qty_cat"].astype(cat_type)

Amazon_Data["currency"].value_counts()
Amazon_Data["currency"] = Amazon_Data["currency"].astype("string")

Amazon_Data["amount"].value_counts()

Amazon_Data = Amazon_Data.drop(columns=["qty"])

In [ ]:
!pip install rapidfuzz
Amazon_Data["ship-city"].value_counts()

# Basic text cleaning (e.g., trimming extra spaces)
cities = Amazon_Data["ship-city"].astype("string").str.strip()

# Count number of words in each city name
word_counts = cities.str.split().str.len()

# Cities with more than 2 words
multi_word_cities = cities[word_counts > 2].unique()

print("Total cities with more than 2 words:", len(multi_word_cities))
print(multi_word_cities[:20])  # show first 20 examples

# Install if needed:
# !pip install rapidfuzz

import re
import pandas as pd
from rapidfuzz import fuzz, process

# === 0) Choose source column and make a cleaned working copy ===
src_col = "ship-city_clean" if "ship-city_clean" in Amazon_Data.columns else "ship-city"
assert src_col in Amazon_Data.columns, f"Column '{src_col}' not found"

def basic_clean(s: pd.Series) -> pd.Series:
    s = s.astype("string")
    s = (s.str.upper()
           .str.replace(r"\(.*?\)", "", regex=True)   # remove content inside parentheses
           .str.replace(r"[^A-Z\s]", " ", regex=True) # keep only letters/spaces
           .str.replace(r"\s+", " ", regex=True)      # collapse multiple spaces
           .str.strip()
         )
    s = s.replace("", pd.NA)
    return s

work = basic_clean(Amazon_Data[src_col])

# Optionally remove generic suffix words (extend this list as needed)
stopwords = [
    "DISTRICT", "TOWNSHIP", "AREA", "INDUSTRIAL", "COLLEGE", "POLICE", "STATION",
    "P O", "PO", "P S", "TEHSIL", "VILLAGE", "URBAN"
]
for w in stopwords:
    # remove the full word; then re-collapse spaces
    work = work.str.replace(rf"\b{w}\b", "", regex=True).str.replace(r"\s+", " ", regex=True).str.strip()

# Optional manual aliases (normalize known multi-word variants)
alias_map = {
    "BOKARO STEEL CITY": "BOKARO",
    "VASCO DA GAMA": "VASCO",
    "GREATER NOIDA WEST": "GREATER NOIDA",
    "SANGLI MIRAJ KUPWAD": "SANGLI",
    "DEHRI ON SONE": "DEHRI",
}
work = work.replace(alias_map)

# === 1) Prepare unique city list and frequencies ===
vc = work.value_counts(dropna=False)
unique_cities = [c for c in vc.index.tolist() if pd.notna(c)]
# Sort by frequency (descending) so common spellings become cluster heads
unique_cities_sorted = sorted(unique_cities, key=lambda x: vc[x], reverse=True)

# === 2) Blocking to speed up comparisons (same first letter + coarse length bucket) ===
def make_block_key(city: str):
    # block by first letter and approximate length bucket
    return (city[0], len(city)//3)

from collections import defaultdict
blocks = defaultdict(list)
for c in unique_cities_sorted:
    blocks[make_block_key(c)].append(c)

# === 3) Fuzzy clustering:
# Use token_set_ratio (robust to token order/extras). Tune threshold if needed.
THRESHOLD = 90  # 0..100; start with 90, lower to 85 if too strict

canonical_map = {}   # variant -> canonical (cluster head)
visited = set()

for block_key, city_list in blocks.items():
    # iterate by frequency (ensure local ordering too)
    city_list = sorted(city_list, key=lambda x: vc[x], reverse=True)
    for head in city_list:
        if head in visited:
            continue
        # start a new cluster with this head
        visited.add(head)
        canonical_map[head] = head

        # Compare only within the same block
        # process.extract returns a list of (match, score, idx)
        matches = process.extract(
            head,
            [c for c in city_list if c != head and c not in visited],
            scorer=fuzz.token_set_ratio,
            score_cutoff=THRESHOLD,
            limit=None
        )

        for match, score, _ in matches:
            visited.add(match)
            # Map the rarer variant to the more frequent head
            canonical_map[match] = head

# Any city not visited (edge cases) maps to itself
for c in unique_cities_sorted:
    canonical_map.setdefault(c, c)

# === 4) Apply mapping back to the DataFrame ===
Amazon_Data["ship-city_fuzzy"] = work.map(canonical_map)

# === 5) Optional: bucket rare cities (<50 rows) into "OTHER" after fuzzy normalization
MIN_COUNT = 50
post_vc = Amazon_Data["ship-city_fuzzy"].value_counts()
rare = post_vc[post_vc < MIN_COUNT].index
Amazon_Data["ship-city_norm"] = Amazon_Data["ship-city_fuzzy"].replace(rare, "OTHER").astype("category")

# === 6) Report summary
before_unique = Amazon_Data[src_col].nunique(dropna=True)
after_unique  = Amazon_Data["ship-city_norm"].nunique(dropna=True)
collapsed     = int((Amazon_Data["ship-city_norm"] == "OTHER").sum())

print({
    "unique_before": int(before_unique),
    "unique_after_fuzzy_no_other": int(post_vc.shape[0]),
    "unique_after_with_OTHER": int(after_unique),
    "rows_collapsed_to_OTHER": collapsed,
    "threshold": THRESHOLD
})

print("\nTop 15 cities after normalization (including OTHER):")
print(Amazon_Data["ship-city_norm"].value_counts().head(15))

# === 7) Inspect a few clusters (helper to review mappings) ===
def show_cluster(head: str, n=10):
    # print top-n original variants mapped to the given head
    variants = [k for k, v in canonical_map.items() if v == head]
    dfv = pd.Series({v: vc.get(v, 0) for v in variants}).sort_values(ascending=False)
    return dfv.head(n)

# Examples (uncomment and adjust)
# display(show_cluster("MUMBAI", 15))
# display(show_cluster("BENGALURU", 15))
# display(show_cluster("GREATER NOIDA", 15))

# Drop intermediate columns not needed further
Amazon_Data = Amazon_Data.drop(columns=["ship-city","ship-city_fuzzy"])


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 24.3 MB/s eta 0:00:00
סה״כ ערים עם יותר מ-2 מילים: 1002
<StringArray>
[                          'BOKARO STEEL CITY',
               'ARANI TIRUVANNAMALAI DISTRICT',
                         'NTPC SIPAT TOWNSHIP',
                    'ORAGADAM INDUSTRIAL AREA',
                     'Calicut medical college',
                    'LAKHIPUR CACHAR DISTRICT',
    'Perole,Nileshwar P.O, kasaragod district',
                       'Gurgaon, HSIIDC SOHNA',
                          'Greater Noida West',
 'Police station - BIKRAM     Tehsil- Danapur',
                          'LILONG IMPHAL WEST',
                    'Pub Daniram Pather,Hojai',
                           'edapally north po',
                     'OCL INDUSTRIAL TOWNSHIP',
                               'VASCO DA GAMA',
                           'Nerul Navi Mumbai',
                    'Armori, Dist. Gadchiroli',
                         'SANGLI MIRAJ KUPWAD',
           

In [ ]:
# Review 'ship-state' distribution before cleaning
Amazon_Data["ship-state"].value_counts()

# Convert to string type and remove leading/trailing spaces
Amazon_Data["ship-state"] = (
    Amazon_Data["ship-state"]
    .astype("string")
    .str.strip()
)

# Convert all values to uppercase for consistency
Amazon_Data["ship-state"] = Amazon_Data["ship-state"].str.upper()

# Define invalid placeholders to be replaced with NaN (Null)
invalid_values = ["", "nan", "na", None, "Null", "N/A"]

# Replace invalid or empty entries with proper NA values
Amazon_Data["ship-state"] = Amazon_Data["ship-state"].replace(invalid_values, pd.NA)


In [ ]:
# Display initial distribution of 'ship-country', including NaN values
print(Amazon_Data["ship-country"].value_counts(dropna=False))

# Ensure the column is of string type for proper manipulation
Amazon_Data["ship-country"] = Amazon_Data["ship-country"].astype("string")

# --- Conditional update ---
# If 'ship-country' is missing but 'months' is present → assign 'INTERNATIONAL'
mask = Amazon_Data["ship-country"].isna() & Amazon_Data["months"].notna()
Amazon_Data.loc[mask, "ship-country"] = "INTERNATIONAL"
# ---------------------------

# Convert 'ship-country' to categorical type for memory efficiency
Amazon_Data["ship-country"] = Amazon_Data["ship-country"].astype("category")

# Display updated value distribution and data type
print(Amazon_Data["ship-country"].value_counts(dropna=False))
print("dtype:", Amazon_Data["ship-country"].dtype)


ship-country
IN     128969
NaN      1457
Name: count, dtype: int64
ship-country
IN               128969
INTERNATIONAL      1424
<NA>                 33
Name: count, dtype: int64
dtype: category


In [ ]:
# Review 'ship-postal-code' values before conversion
Amazon_Data["ship-postal-code"].value_counts()

# Convert postal codes to numeric (non-numeric values become NaN)
Amazon_Data["ship-postal-code"] = pd.to_numeric(
    Amazon_Data["ship-postal-code"], errors="coerce"
)

# Count how many values became NaN after conversion
n_null = Amazon_Data["ship-postal-code"].isna().sum()

# Convert to nullable integer type (Int64 supports NA values)
Amazon_Data["ship-postal-code"] = Amazon_Data["ship-postal-code"].astype("Int64")

# Display final data type and sample distribution
print(Amazon_Data["ship-postal-code"].dtypes)
Amazon_Data["ship-postal-code"].value_counts()


Int64


,count
ship-postal-code,
201301,1006
122001,688
560037,633
560068,619
560076,581
...,...
241204,1
382029,1
524228,1


In [ ]:
# Review 'promotion-ids' distribution before transformation
Amazon_Data["promotion-ids"].value_counts()

# Create a Boolean column indicating whether a promotion exists
Amazon_Data["has_promotion"] = Amazon_Data["promotion-ids"].notna()

# Quick validation of new column values
print(Amazon_Data["has_promotion"].value_counts())

# Drop the original 'promotion-ids' column after extraction
Amazon_Data = Amazon_Data.drop(columns=["promotion-ids"])


has_promotion
True     79840
False    50586
Name: count, dtype: int64


In [ ]:
# Review initial distribution of 'b2b' column
Amazon_Data["b2b"].value_counts()

# Convert 'b2b' column to boolean type (supports True/False/NA)
Amazon_Data["b2b"] = Amazon_Data["b2b"].astype("boolean")

# Display value counts including NaN values
Amazon_Data["b2b"].value_counts(dropna=False)


,count
b2b,
False,128131
<NA>,1424
True,871


In [ ]:
# Review distributions of both fulfilment-related columns
Amazon_Data["fulfilled-by"].value_counts()
Amazon_Data["fulfilment"].value_counts()

# --- 1) Basic logical conditions ---
is_merchant = Amazon_Data["fulfilment"].astype("string").str.strip().str.lower() == "merchant"
is_amazon   = Amazon_Data["fulfilment"].astype("string").str.strip().str.lower() == "amazon"
is_easyship = Amazon_Data["fulfilled-by"].astype("string").str.strip().str.lower() == "easy ship"
is_null_ship = Amazon_Data["fulfilled-by"].isna() | (Amazon_Data["fulfilled-by"].astype("string").str.strip() == "")

# --- 2) Check if all "Merchant" rows are also "Easy Ship" ---
merchant_total = is_merchant.sum()
merchant_easyship = (is_merchant & is_easyship).sum()

if merchant_total == merchant_easyship:
    print("✅ All Merchant rows match Easy Ship exactly.")
else:
    mismatch = Amazon_Data[is_merchant & ~is_easyship]
    print(f"⚠️ Found {len(mismatch)} Merchant rows that are NOT Easy Ship.")
    # 'mismatch' will display the exception rows for review

# --- 3) Handle cases where fulfilment='Amazon' but 'fulfilled-by' is missing ---
amazon_null_ship = Amazon_Data[is_amazon & is_null_ship]
if len(amazon_null_ship) > 0:
    print(f"⚠️ Found {len(amazon_null_ship)} rows with fulfilment=Amazon but missing fulfilled-by. Converting to Merchant.")
    # Update missing fulfilled-by values for these rows to "Merchant"
    Amazon_Data.loc[is_amazon & is_null_ship, "fulfilment"] = "Merchant"

# --- 4) Create a Boolean column marking Easy Ship sellers ---
Amazon_Data["seller_easy_ship"] = (
    Amazon_Data["fulfilled-by"]
    .astype("string")
    .str.strip()
    .str.lower()
    .eq("easy ship")
    .fillna(False)   # fill NaN with False
    .astype(bool)    # convert to regular bool type
)

# Review final distribution including any False/True counts
Amazon_Data["seller_easy_ship"].value_counts(dropna=False)


✅ All Merchant rows match Easy Ship exactly.
⚠️ Found 89716 rows with fulfilment=Amazon but missing fulfilled-by. Converting to Merchant.


,count
seller_easy_ship,
False,91140
True,39286


In [ ]:
# Review the distribution of the column 'unnamed:_22'
Amazon_Data["unnamed:_22"].value_counts()

# Print unique values to inspect potential hidden data or artifacts
print(Amazon_Data["unnamed:_22"].unique())

# Drop the redundant or empty column
Amazon_Data = Amazon_Data.drop(columns=["unnamed:_22"])


[nan False]


In [ ]:
Amazon_Data["sku_clean"].value_counts()



,count
sku_clean,
JNE3797-KR-L,773
JNE3797-KR-M,657
JNE3797-KR-S,587
JNE3405-KR-L,535
J0230-SKD-M,507
...,...
JNE2323-KR-574-XS,1
J0314-KR-L,1
AN212-WHITE-L,1


In [ ]:
# Review 'date_parsed' values before removal (for validation)
Amazon_Data["date_parsed"].value_counts()

# Drop the 'date_parsed' column (not needed since 'date' already contains this info)
Amazon_Data = Amazon_Data.drop(columns=["date_parsed"])


In [ ]:
# Clean and normalize the 'weekend' column as string (lowercase, trimmed)
s = Amazon_Data["weekend"].astype("string").str.strip().str.lower()

# Fill missing values with False and convert to boolean type
Amazon_Data["weekend"] = Amazon_Data["weekend"].fillna(False).astype(bool)


In [ ]:
# Drop the redundant 'months' column (information already stored in 'month')
Amazon_Data = Amazon_Data.drop(columns=["months"])


In [ ]:
# Review the original distribution of 'customer' values
Amazon_Data["customer"].value_counts()

# Clean the 'customer' column: trim spaces and convert to lowercase
Amazon_Data["customer_clean"] = (
    Amazon_Data["customer"]
    .astype("string")
    .str.strip()
    .str.lower()
)

# Display frequency of cleaned customer names
print(Amazon_Data["customer_clean"].value_counts())

# Drop the original 'customer' column (no longer needed)
Amazon_Data = Amazon_Data.drop(columns=["customer"])


customer_clean
mulberries boutique                269
jeannie thein win                  104
diya designs                        84
gajanan                             83
chaaya sookdeb                      73
avin                                54
ria fashion                         53
priyanka gupta                      51
fusion fashions corp. (gopikas)     50
valli arangan fashion               48
the pink market                     46
vinodha                             41
kaveenaz collection                 33
amandeep kaur                       30
mrs sumitha raju                    30
nitha pushpangadan sarala           29
siva                                29
r. vikneswari                       29
rakshanda mahajan                   29
bhanu saleine naunitham             26
g janani                            23
kamala rengasamy                    23
babitha huxley                      22
tanusree majumder                   21
swati reddy                         17
rino sanda

In [ ]:
# Review the distribution of 'pcs' values before cleaning
Amazon_Data["pcs"].value_counts()

# Fill missing values with 0 and convert the column to integer type
Amazon_Data["pcs"] = Amazon_Data["pcs"].fillna(0).astype(int)

# Verify data type and value distribution after conversion
print(Amazon_Data["pcs"].dtypes)
print(Amazon_Data["pcs"].value_counts())


int64
pcs
0    129002
1      1335
2        40
5        34
3        14
4         1
Name: count, dtype: int64


In [ ]:
# Review value counts for 'sku_new' before conversion
Amazon_Data["sku_new"].value_counts()

# Convert 'sku_new' column to string type
Amazon_Data["sku_new"] = Amazon_Data["sku_new"].astype("string")

# Review value counts for 'sku_code' before conversion
Amazon_Data["sku_code"].value_counts()

# Convert 'sku_code' column to string type
Amazon_Data["sku_code"] = Amazon_Data["sku_code"].astype("string")

# Review value counts for 'design_no.' before conversion
Amazon_Data["design_no."].value_counts()

# Convert 'design_no.' column to string type
Amazon_Data["design_no."] = Amazon_Data["design_no."].astype("string")


In [ ]:
Amazon_Data["category_stock"].value_counts()

import re

# --- 1) Base: standardize textual cleaning ---
s = (
    Amazon_Data["category_stock"]
    .astype("string")
    .str.upper()
    .str.replace(r"[^A-Z\s]", " ", regex=True)  # remove symbols like ":" or others
    .str.replace(r"\s+", " ", regex=True)       # collapse multiple spaces
    .str.strip()
)

# --- 2) Category normalization rules (regex-based) ---
def normalize_category(txt: str) -> str:
    if txt is None or pd.isna(txt):
        return txt

    # Common typos / variants → standardized categories
    rules = [
        (r"^KURTI$", "KURTA"),
        (r"^NIGHT\s*WEAR$", "NIGHTWEAR"),
        (r"^LEHENGA\s+CHOLI$", "LEHENGA CHOLI"),
        (r"^PALAZ+O$", "PALAZZO"),               # PALAZO / PLAZZO → PALAZZO
        (r"^PANT(S)?$", "PANTS"),
        (r"^(AN\s+)?LEGGINGS$", "LEGGINGS"),
        (r"^BOTTOM(S)?$", "BOTTOMS"),
        # Sets
        (r"^KURTA\s+SET$", "SET"),               # Merge 'KURTA SET' into 'SET' (can adjust policy)
        (r"^(CROP\s+TOP).*(PALAZZO|PLAZZO)$", "CROP TOP SET"),
    ]

    for pat, repl in rules:
        if re.fullmatch(pat, txt):
            return repl

    # If no match found, return cleaned original text
    return txt

# --- 3) Apply normalization function and create new column ---
Amazon_Data["category_stock_norm"] = s.map(normalize_category)

# --- 4) Count normalized values ---
norm_counts = Amazon_Data["category_stock_norm"].value_counts(dropna=False)

# --- 5) Compare before/after normalization ---
before = s.value_counts(dropna=False).rename("count_before")
after  = norm_counts.rename("count_after")
compare = (
    pd.concat([before, after], axis=1)
    .fillna(0)
    .astype({"count_before": "int64", "count_after": "int64"})
    .assign(diff=lambda df: df["count_after"] - df["count_before"])
    .sort_values("count_after", ascending=False)
)

# Display results
print("== Normalized value counts ==")
print(norm_counts)

print("\n== Before/After comparison (top rows) ==")
print(compare.head(30))

# Ensure the normalized column is stored as string
Amazon_Data["category_stock_norm"] = Amazon_Data["category_stock_norm"].astype("string")


== Normalized value counts ==
category_stock_norm
KURTA            56677
SET              47237
DRESS            10721
TOP               9008
NIGHTWEAR         2317
TUNIC             1478
BLOUSE             846
LEHENGA CHOLI      518
CROP TOP SET       499
<NA>               282
CROP TOP           201
PALAZZO            187
SAREE              151
PANTS              134
LEGGINGS           124
JUMPSUIT            24
CARDIGAN            21
BOTTOMS              1
Name: count, dtype: int64

== Before/After comparison (top rows) ==
                      count_before  count_after   diff
KURTA                        56646        56677     31
SET                          14040        47237  33197
DRESS                        10721        10721      0
TOP                           9008         9008      0
NIGHTWEAR                        0         2317   2317
TUNIC                         1478         1478      0
BLOUSE                         846          846      0
LEHENGA CHOLI               

In [ ]:
# Review the distribution of 'size_stock' before normalization
Amazon_Data["size_stock"].value_counts()

# Define size normalization mapping
size_map = {
    "m": "M",
    "l": "L",
    "s": "S",
    "xl": "XL",
    "xxl": "XXL",
    "xxxl": "3XL",
    "XXXL": "3XL",
    "S TO XXL": "Free",   # unified as 'Free'
    "6XL": "4XL",
    "5XL": "4XL"
}

# Apply normalization mapping
Amazon_Data["size_stock"] = Amazon_Data["size_stock"].replace(size_map)

# Convert to categorical type for efficiency
Amazon_Data["size_stock"] = Amazon_Data["size_stock"].astype("category")

# Review final value distribution after normalization
Amazon_Data["size_stock"].value_counts()


,count
size_stock,
M,22958
L,22351
XL,21094
XXL,18257
S,17277
3XL,14904
XS,11240
4XL,1790
FREE,277


In [ ]:
Amazon_Data["color"].value_counts()

import re
import pandas as pd

# 1) Basic text cleanup
raw = (
    Amazon_Data["color"]
    .astype("string")
    .str.strip()
)

def normalize_color(v: str):
    if v is pd.NA or v is None:
        return v
    x = re.sub(r"[^A-Za-z\s]", " ", str(v)).upper()     # remove symbols like ':', '_', etc.
    x = re.sub(r"\s+", " ", x).strip()

    # 2) Mapping dictionary for synonyms / variations
    syn = {
        "OFF WHITE": "White",
        "OFFWHITE": "White",
        "IVORY": "White",

        "CREAM": "Cream",  # if you want to merge with Off White, change to "Off White"
        "BEIGE": "Beige",

        "NAVY": "Blue",
        "NAVY BLUE": "Blue",
        "DARK BLUE": "Blue",

        "TEAL BLUE": "Blue",
        "TEAL GREEN": "Green",
        "LIGHT GREEN": "Green",

        "TURQUOISE": "Turquoise",
        "TURQUOISE BLUE": "Turquoise",
        "TURQUOISE GREEN": "Turquoise",

        "LEMON": "Yellow",
        "LEMON YELLOW": "Yellow",

        "MINT": "Green",
        "MINT GREEN": "Green",

        "OLIVE": "Green",
        "OLIVE GREEN": "Green",

        "LIGHT BROWN": "Brown",

        "GREY": "Grey",
        "GRAY": "Grey",

        "BURGUNDY": "Maroon",
        "WINE": "Maroon",

        "MULTI COLOR": "Multicolor",
        "MULTICOLOR": "Multicolor",

        "CORAL": "Coral",
        "CORAL PINK": "Coral",
        "CORAL ORANGE": "Coral",

        "POWDER BLUE": "Blue",
        "SKY BLUE": "Blue",
        "LIGHT BLUE": "Blue",

        "CHARCOAL": "Brown",
        "BROWN": "Brown",
        "CHOCOLATE": "Brown",
        "TAUPE": "Grey",
        "KHAKI": "Yellow",
        "LIGHT YELLOW": "Yellow",
        "LIME GREEN": "Green",
        "LIGHT PINK": "Pink",
        "DARK GREEN": "Green",
        "SEA GREEN": "Green",
        "CHIKU": "Beige"
    }

    # 3) Direct dictionary match
    if x in syn:
        return syn[x]

    # 4) Pattern for shades: LIGHT/DARK + base color
    m = re.match(r"^(LIGHT|DARK)\s+(.*)$", x)
    if m:
        shade, base = m.groups()
        base = base.strip()
        base_std = syn.get(base, base.title())
        return f"{shade.title()} {base_std}"

    # 5) Default: Title Case
    return x.title()

# 6) Apply normalization and create new column
Amazon_Data["color_norm"] = raw.map(normalize_color)

# Count normalized values
counts_after = Amazon_Data["color_norm"].value_counts(dropna=False)

# 7) Comparison table (before/after)
counts_before = raw.value_counts(dropna=False).rename("count_before")
compare = (
    pd.concat([counts_before, counts_after.rename("count_after")], axis=1)
    .fillna(0).astype(int)
    .assign(diff=lambda df: df["count_after"] - df["count_before"])
    .sort_values("count_after", ascending=False)
)

print("== Normalized value counts ==")
print(counts_after.head(50))
print("\n== Before/After comparison (top 50) ==")
print(compare.head(50))


== Normalized value counts ==
color_norm
Green         22568
Blue          19934
Pink          12943
White          9408
Mustard        8686
Black          7930
Teal           7372
Peach          6545
Maroon         5521
Grey           4366
Red            3515
Turquoise      3221
Orange         3044
Yellow         2951
Cream          2617
Brown          2108
Multicolor     1778
Magenta        1407
Beige          1197
Gold            941
Coral           515
Mauve           500
Rust            408
Indigo          357
Purple          312
<NA>            282
Name: count, dtype: int64

== Before/After comparison (top 50) ==
                 count_before  count_after  diff
Green                   13385        22568  9183
Blue                    14507        19934  5427
Pink                    12004        12943   939
White                    5480         9408  3928
Mustard                  8686         8686     0
Black                    7930         7930     0
Teal                     7372 

In [ ]:
# Review normalized color distribution
Amazon_Data["color_norm"].value_counts()

# Convert 'color_norm' column to string type for consistency
Amazon_Data["color_norm"] = Amazon_Data["color_norm"].astype("string")

# Drop the original 'color' column (no longer needed after normalization)
Amazon_Data = Amazon_Data.drop(columns=["color"])


In [ ]:
# Review 'fulfilment' distribution before conversion
Amazon_Data["fulfilment"].value_counts()

# Convert 'fulfilment' column to categorical type for memory efficiency
Amazon_Data["fulfilment"] = Amazon_Data["fulfilment"].astype("category")


In [ ]:
# Review 'category_stock' value distribution before dropping
Amazon_Data["category_stock"].value_counts()

# Drop the original 'category_stock' column (after normalization was completed)
Amazon_Data = Amazon_Data.drop(columns=["category_stock"])


In [ ]:
# Review the distribution of 'sku_id_stock' before dropping
Amazon_Data["sku_id_stock"].value_counts()

# Drop the 'sku_id_stock' column (no longer needed)
Amazon_Data = Amazon_Data.drop(columns=["sku_id_stock"])


In [ ]:
# Review the distribution of 'sku_size_stock' before dropping
Amazon_Data["sku_size_stock"].value_counts()

# Drop the 'sku_size_stock' column (no longer needed)
Amazon_Data = Amazon_Data.drop(columns=["sku_size_stock"])


In [ ]:
# Rename the column 'design_no.' to 'design_no' for consistency
Amazon_Data.rename(columns={"design_no.": "design_no"}, inplace=True)


In [ ]:
# Build a summary table describing each column in the DataFrame
summary = pd.DataFrame({
    "column": Amazon_Data.columns,                  # column names
    "dtype": Amazon_Data.dtypes.values,             # data types
    "null_count": Amazon_Data.isnull().sum().values, # number of missing values per column
    "unique_count": Amazon_Data.nunique().values     # number of unique values per column
})

# Display the summary table
print(summary)


                  column           dtype  null_count  unique_count
0                  index           int64           0        128975
1               order_id  string[python]        1424        120378
2             fulfilment        category        1424             2
3     ship-service-level        category        1424             2
4               category        category        1424             8
5                   size        category           0             9
6                   asin  string[python]        1424          7190
7         courier_status        category        8296             3
8               currency  string[python]        9222             1
9                 amount         float64        9222          1410
10            ship-state  string[python]        1457            47
11      ship-postal-code           Int64        1457          9459
12          ship-country        category          33             2
13                   b2b         boolean        1424          

In [ ]:
# Save the cleaned and processed DataFrame to a Pickle file
Amazon_Data.to_pickle("best_dataset_clean_2.pkl")


In [ ]:
# Drop unnecessary columns before performing EDA (Exploratory Data Analysis)
import pandas as pd

# Load the cleaned dataset from pickle
df = pd.read_pickle("best_dataset_clean_2.pkl")

# Remove columns that are not required for analysis
df = df.drop(columns=["index", "order_id", "sku_id_size", "sku_clean", "sku_new", "sku_code", "style_id"])


In [ ]:
# Review current category distribution
df["category"].value_counts()

import pandas as pd

# Map old category names to new standardized ones
category_map = {
    "Ethnic Dress": "kurta",
    "Blouse": "Top",
    "Saree": "kurta",
    "Bottom": "Western Dress"
}

# Apply category mapping and convert to string
df["category"] = (
    df["category"]
    .astype("string")
    .replace(category_map)
)

# Convert back to categorical type for efficiency
df["category"] = df["category"].astype("category")

# Verify the updated category distribution
print(df["category"].value_counts())


category
kurta            51230
Set              50284
Western Dress    15940
Top              11548
Name: count, dtype: int64


In [ ]:
# Display the current distribution of courier status values
df["courier_status"].value_counts()


,count
courier_status,
Shipped,109510
Unshipped,6682
Cancelled,5938


In [ ]:
# Display value counts for the 'status_first_word' column  (TARGET)
df["status_first_word"].value_counts()



,count
status_first_word,
Shipped,109719
Cancelled,18336
Pending,939
Shipping,8


In [ ]:
# Normalize 'status_first_word' values by merging similar categories
df["status_first_word"] = (
    df["status_first_word"]
    .astype("string")
    .replace({"Cancelled": "Shipped", "Shipping": "Shipped"})
)

# Convert to categorical type for optimized storage
df["status_first_word"] = df["status_first_word"].astype("category")

# Display updated status value distribution
df["status_first_word"].value_counts()

# Rename the column for clarity
df.rename(columns={'status_first_word': 'status'}, inplace=True)


In [ ]:
df.status.value_counts()


,count
status,
Shipped,128063
Pending,939


In [ ]:
import pandas as pd
import numpy as np

# Function to map month numbers to corresponding seasons
def map_month_to_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Summer"
    elif month in [6, 7, 8, 9]:
        return "Monsoon"
    elif month in [10, 11]:
        return "Post-Monsoon"
    else:
        return np.nan  # handle missing or invalid month values

# Apply the function to the 'month' column
df["season"] = df["month"].apply(map_month_to_season)

# Convert to categorical type for optimized memory usage
df["season"] = df["season"].astype("category")

# Verify distribution of season categories
print(df["season"].value_counts())


season
Summer     92727
Monsoon    37699
Name: count, dtype: int64


In [ ]:
# Ensure 'ship-city_norm' column is stored as string type
df["ship-city_norm"] = df["ship-city_norm"].astype("string")



In [ ]:
# Drop unnecessary columns 'year' and 'rate' from the DataFrame
df.drop(columns=["year", "rate"], inplace=True)


In [ ]:
import pandas as pd

# Convert numeric weekday values to weekday names
weekday_map = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday"
}
df["weekday"] = df["weekday"].map(weekday_map).astype("category")

# Convert numeric month values to month names
month_map = {
    1: "January", 2: "February", 3: "March",
    4: "April",   5: "May",      6: "June",
    7: "July",    8: "August",   9: "September",
    10: "October",11: "November",12: "December"
}
df["month"] = df["month"].map(month_map).astype("category")


In [ ]:
# Clean extra spaces / inconsistent casing if needed
# df["courier_status"] = df["courier_status"].astype(str).str.strip()

# If pcs = 0 and courier_status = "Shipped" → set pcs = 1
mask_shipped = (df["pcs"] == 0) & (df["courier_status"].str.lower() == "shipped")
df.loc[mask_shipped, "pcs"] = 1

# If courier_status = "Unshipped" → set pcs = 0
mask_unshipped = df["courier_status"].str.lower() == "unshipped"
df.loc[mask_unshipped, "pcs"] = 0

# Convert pcs column to integer type
df["pcs"] = df["pcs"].astype(int)

# Quick verification
print(df["pcs"].value_counts())


pcs
1    110845
0     19492
2        40
5        34
3        14
4         1
Name: count, dtype: int64


In [ ]:
# Recalculate gross_amt based on pcs * amount
df["gross_amt_calc"] = df["pcs"] * df["amount"]

# Replace missing or zero gross_amt values with the calculated ones
df["gross_amt"] = np.where(
    (df["gross_amt"].isna()) | (df["gross_amt"] == 0),
    df["gross_amt_calc"],
    df["gross_amt"]
)

# Drop the temporary calculation column if no longer needed
df.drop(columns=["gross_amt_calc"], inplace=True)

# Verify results
print(df[["pcs", "amount", "gross_amt"]].head(10))


   pcs  amount  gross_amt
0    1   761.0      761.0
1    0     NaN        NaN
2    1   519.0      519.0
3    1  1165.0     1165.0
4    1   358.0      358.0
5    1   417.0      417.0
6    1   688.0      688.0
7    1   657.0      657.0
8    1   453.0      453.0
9    1   599.0      599.0


In [ ]:
# Define ordered categories for 'pcs' column based on existing values
df['pcs'] = pd.Categorical(df['pcs'], categories=[0, 1, 2, 3, 4, 5], ordered=True)

# Check the data type to confirm conversion
df['pcs'].dtype

# Display defined category order
df['pcs'].cat.categories


Index([0, 1, 2, 3, 4, 5], dtype='int64')

In [ ]:
import pandas as pd
import numpy as np


# =======================================================
# 1. DataFrame normalization: targeted handling of missing values (Null)
# =======================================================
df_cleaned = df.copy()

# A. Replace pd.NA and ±inf with np.nan across the entire frame
#    This avoids errors like 'boolean value of NA is ambiguous'.
df_cleaned = df_cleaned.replace(pd.NA, np.nan)
df_cleaned = df_cleaned.replace([np.inf, -np.inf], np.nan)

# B. Targeted handling by dtype
for col in df_cleaned.columns:

    # 1) Numeric columns (float, int, Int64)
    #    i = integer, u = unsigned, f = float → cast to float64 for uniformity
    if df_cleaned[col].dtype.kind in 'iuf':
        df_cleaned[col] = df_cleaned[col].astype(np.float64)

    # 2) String/Categorical columns (object, category, string[python])
    elif df_cleaned[col].dtype.kind in 'O' or 'string' in str(df_cleaned[col].dtype) or 'category' in str(df_cleaned[col].dtype):

        # a) Choose a default fill value for NA/NaN
        fill_val = 'MISSING_CATEGORY'
        if col == "courier_status":
             fill_val = 'UNKNOWN_STATUS'

        # b) Convert to str and fillna BEFORE converting to category
        #    This ensures we don’t operate on a locked categorical dtype.
        #    Assumption: all text columns should be treated as categories for EDA/AutoViz.
        df_cleaned[col] = df_cleaned[col].astype(str).fillna(fill_val)

        # c) Convert to 'category' so EDA tools/Pandas won’t attempt numeric correlations
        df_cleaned[col] = df_cleaned[col].astype('category')

    # 3) Boolean columns (boolean, bool)
    elif df_cleaned[col].dtype.kind in 'b':
        # Fill missing values (if any) with False by default
        df_cleaned[col] = df_cleaned[col].fillna(False)


In [ ]:
# Build a summary table showing column metadata
summary = pd.DataFrame({
    "column": df.columns,                   # column names
    "dtype": df.dtypes.values,              # data types
    "null_count": df.isnull().sum().values, # number of missing values
    "unique_count": df.nunique().values     # number of unique values
})

# Display the summary table
print(summary)


                  column           dtype  null_count  unique_count
0             fulfilment        category        1424             2
1     ship-service-level        category        1424             2
2               category        category        1424             4
3                   size        category           0             9
4                   asin  string[python]        1424          7190
5         courier_status        category        8296             3
6               currency  string[python]        9222             1
7                 amount         float64        9222          1410
8             ship-state  string[python]        1457            47
9       ship-postal-code           Int64        1457          9459
10          ship-country        category          33             2
11                   b2b         boolean        1424             2
12                   day           int64           0            31
13                 month        category           0          

In [ ]:
# Save the updated and cleaned DataFrame to a new Pickle file
df.to_pickle("best_dataset_clean_2.1.pkl")
